In [8]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import numpy as np
import cv2
import os
import glob

%matplotlib inline
 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

square_side_length = 23.0 # チェスボード内の正方形の1辺のサイズ(mm)
grid_intersection_size = (9, 6) # チェスボード内の格子数

pattern_points = np.zeros( (np.prod(grid_intersection_size), 3), np.float32 )
pattern_points[:,:2] = np.indices(grid_intersection_size).T.reshape(-1, 2)
pattern_points *= square_side_length
object_points = []
image_points = []

camera_mat, dist_coef = [], []

In [9]:
images = glob.glob('./wide_angle_chessboard/*.jpg')
print(len(images))

28


In [10]:
for img_path in images:

        frame = cv2.imread(img_path)
        # チェスボード検出用にグレースケール画像へ変換
        grayscale_image = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

        # チェスボードのコーナーを検出
        found, corner = cv2.findChessboardCorners(grayscale_image, grid_intersection_size)
        #found, corner = cv2.findChessboardCorners(frame, grid_intersection_size)
        if found == True:
            print('findChessboardCorners : True')

            # 現在のOpenCVではfindChessboardCorners()内で、cornerSubPix()相当の処理が実施されている？要確認
            #term = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_COUNT, 30, 0.1)
            #cv2.cornerSubPix(grayscale_image, corner, (5,5), (-1,-1), term)
            #cv2.drawChessboardCorners(grayscale_image, grid_intersection_size, corner, found)
            image_points.append(corner)
            object_points.append(pattern_points)

            
        if found == False:
            print('findChessboardCorners : False')

findChessboardCorners : True
findChessboardCorners : True
findChessboardCorners : True
findChessboardCorners : True
findChessboardCorners : True
findChessboardCorners : True
findChessboardCorners : True
findChessboardCorners : True
findChessboardCorners : True
findChessboardCorners : True
findChessboardCorners : True
findChessboardCorners : True
findChessboardCorners : True
findChessboardCorners : True
findChessboardCorners : False
findChessboardCorners : True
findChessboardCorners : True
findChessboardCorners : True
findChessboardCorners : True
findChessboardCorners : True
findChessboardCorners : True
findChessboardCorners : True
findChessboardCorners : True
findChessboardCorners : True
findChessboardCorners : True
findChessboardCorners : True
findChessboardCorners : True
findChessboardCorners : True


In [11]:
if len(image_points) > 0:
        # カメラ内部パラメータを計算
        print('calibrateCamera() start')
        rms, K, d, r, t = cv2.calibrateCamera(object_points,image_points,(frame.shape[1],frame.shape[0]),None,None)
        #rms, K, d, r, t = cv2.fisheye.calibrate(object_points,image_points,(frame.shape[1],frame.shape[0]),None,None)
        print("RMS = ", rms)
        print("K = \n", K)
        print("d = ", d.ravel())
        np.savetxt("K-chessboard.csv", K, delimiter =',',fmt="%0.14f") #カメラ行列の保存
        np.savetxt("d-chessboard.csv", d, delimiter =',',fmt="%0.14f") #歪み係数の保存

        camera_mat = K
        dist_coef = d

        # 再投影誤差による評価
        mean_error = 0
        for i in range(len(object_points)):
            image_points2, _ = cv2.projectPoints(object_points[i], r[i], t[i], camera_mat, dist_coef)
            error = cv2.norm(image_points[i], image_points2, cv2.NORM_L2) / len(image_points2)
            mean_error += error
        print("total error: ", mean_error/len(object_points)) # 0に近い値が望ましい(魚眼レンズの評価には不適？)

calibrateCamera() start
RMS =  0.6306923527019362
K = 
 [[512.27702586   0.         323.20273502]
 [  0.         510.45949391 232.43226068]
 [  0.           0.           1.        ]]
d =  [-0.40665648  0.21677875  0.00162543 -0.00091249 -0.07384577]
total error:  0.0810888370108942
